In [23]:
import numpy as np
from data import Vertebral_column
from data import Co_Author
from Processing_Data import Abanole
from Processing_Data import Ecoli
from Processing_Data import Ecloli1
from Processing_Data import Ecoli3
from Processing_Data import Glass1
from Processing_Data import Glass4
from Processing_Data import Haberman
from Processing_Data import Waveform
from Processing_Data import New_thyroid2
from Processing_Data import Page_blocks
from Processing_Data import Pima_Indians_Diabetes
from Processing_Data import Satimage
from Processing_Data import Transfusion
from Processing_Data import Yeast
from data import indian_liver_patient
#from data import spect_heart
from wsvm.application import Wsvm
from svm.application import Svm
from sklearn.svm import SVC
#from sklearn.metrics import f1_score
from sklearn.metrics  import classification_report,precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score, confusion_matrix,roc_auc_score,f1_score
from sklearn.neighbors import NearestNeighbors
from sklearn.utils import _safe_indexing
from sklearn import metrics
import math
from datetime import datetime
from fuzzy.weight import fuzzy

In [24]:
def compute_weight(X, y,name_method ="own_class_center_divided", name_function = "lin_center_own", beta = None,C = None, gamma = None, u = None, sigma = None):
    method = fuzzy.method()
    function = fuzzy.function()
    pos_index = np.where(y == 1)[0]
    neg_index = np.where(y == -1)[0]
    try:
        if name_method == "own_class_center": 
            d = method.own_class_center(X, y)
        elif name_method == "estimated_hyper_lin": # actual_hyper_lin, own_class_center
            d = method.estimated_hyper_lin(X, y)
        elif name_method == "own_class_center_opposite":
            d = method.own_class_center_opposite(X, y)
        elif name_method == 'actual_hyper_lin':
            d = method.actual_hyper_lin(X, y,C = C, gamma = gamma)
        elif name_method == 'own_class_center_divided':
            print("AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA")
            d = method.own_class_center_divided(X, y)
        else:
            print('dont exist method')
        
        if name_function == "lin":
            W = function.lin(d)
        elif name_function == "exp":
            W = function.exp(d, beta)
        elif name_function == "lin_center_own":
            W = function.lin_center_own(d, pos_index,neg_index)
        elif name_function == 'gau':
            W = function.gau(d, u, sigma)
    except Exception as e:
        print('dont exist function')
        print(e)
    pos_index = np.where(y == 1)[0]
    neg_index = np.where(y == -1)[0]
    r_pos = 1
    r_neg = len(pos_index)/len(neg_index)
    m = []
    W = np.array(W)
    m = W[pos_index]*r_pos
    m = np.append(m, W[neg_index]*r_neg)
    return m

In [25]:
def wsvm(C,X_train, y_train,X_test,distribution_weight=None):
    model = Wsvm(C,distribution_weight)
    model.fit(X_train, y_train)
    test_pred = model.predict(X_test)
    return test_pred

In [26]:
def Gmean(y_test,y_pred):
    cm_WSVM = metrics.confusion_matrix(y_test, y_pred)
    sensitivity = cm_WSVM[1,1]/(cm_WSVM[1,0]+cm_WSVM[1,1])
    specificity = cm_WSVM[0,0]/(cm_WSVM[0,0]+cm_WSVM[0,1])
    gmean = math.sqrt(sensitivity*specificity)
    return sensitivity,specificity,gmean

In [27]:
def is_tomek(X,y, class_type):
    nn = NearestNeighbors(n_neighbors=2)
    nn.fit(X)
    nn_index = nn.kneighbors(X, return_distance=False)[:, 1]
    links = np.zeros(len(y), dtype=bool)
    # find which class to not consider
    class_excluded = [c for c in np.unique(y) if c not in class_type]
    X_dangxet = []
    X_tl = []
    # there is a Tomek link between two samples if they are both nearest
    # neighbors of each others.
    for index_sample, target_sample in enumerate(y):
        if target_sample in class_excluded:
            continue
        if y[nn_index[index_sample]] != target_sample:
            if nn_index[nn_index[index_sample]] == index_sample:
                X_tl.append(index_sample)
                X_dangxet.append(nn_index[index_sample])
                links[index_sample] = True

    return links,X_dangxet,X_tl

In [28]:
def data_tomelinks(C,X_train_val,y_train_val,X_test_val, y_test_val,n_neighbors,clf=None,namemethod=None,namefunction=None):
    links,xdx,xtl = is_tomek(X_train_val,y_train_val,class_type=[-1.0])
    if clf == None:
        if namemethod == None:
            distribution_weight = compute_weight(X_train_val, y_train_val)
            print("\nW mac dinh\n")
        else:
            if namefunction == "exp":
                distribution_weight = compute_weight(X_train_val, y_train_val,name_method = namemethod,name_function = namefunction,beta = 0.5)
                print("BETA: ",0.5)
            else:
                distribution_weight = compute_weight(X_train_val, y_train_val,name_method = namemethod,name_function = namefunction)
                print("Ko exp")
                #print(distribution_weight)
        clf = Wsvm(C,distribution_weight)
        print(clf)
        clf.fit(X_train_val, y_train_val)
    else:
        clf.fit(X_train_val, y_train_val)
    y_predict = clf.predict(X_test_val)
    sensitivity,specificity,gmean = Gmean(y_test_val,y_predict)
    nn2 = NearestNeighbors(n_neighbors=n_neighbors)
    nn2.fit(X_train_val)
    y_nn = []
    y_check_pos = np.array(y_train_val <-2)
    for ind,i in enumerate(xdx):
        y_pred = clf.predict([X_train_val[i]])  #
        if y_pred == -1.0:                          
            knn_X = (nn2.kneighbors([X_train_val[i]])[1]).tolist() 
            for j in knn_X[0]:
                y_nn.append(y_train_val[j])    # gom nhãn láng giềng của X_train[i] bị dự đoán sai vào y_nn
        else:
            y_check_pos[xtl[ind]] = True

    y_nn = np.array(y_nn)
    if len(y_nn)>0:
        y_nn = np.array_split(y_nn, len(y_nn)/n_neighbors)
    y_check_neg = np.array(y_train_val <-2)
    for i in range(0,len(y_nn)):      #
        if 1 not in y_nn[i][1:]:      # Nếu không có nhãn 1 xung quanh X_train[i] bị dự đoán sai => xóa X_train[i]
            y_check_neg[xdx[i]] = True

    y_check = y_check_neg | y_check_pos
    
    sample_indices_ = np.flatnonzero(np.logical_not(y_check)) 
    ytl = _safe_indexing(y_train_val, sample_indices_)
    Xtl = _safe_indexing(X_train_val, sample_indices_)
    return Xtl,ytl,gmean

In [29]:
def lfb(C,namemethod,namefunction,sample_train_val,T,X_train_val,y_train_val,X_test_val, y_test_val,n_neighbors,thamso1,thamso2): #loop find the best
    gmean = 0
    for i in range(0,T):
        print(f"\t\t Vong Lap thu: T = {i+1}")
        #f.write(f"\n===================================================================================================================")
        print(f"\nSo luong mau training ban dau: {sample_train_val}")
        print(f"\n\n\tFuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
        print(len(X_train_val))
        X_train_val, y_train_val, gmean2 = data_tomelinks(C,X_train_val,y_train_val,X_test_val, y_test_val,n_neighbors,clf=None,namemethod=namemethod,namefunction=namefunction)
        if namefunction == "exp":
            distribution_weight = compute_weight(X_train_val, y_train_val,name_method = namemethod,name_function = namefunction,beta = 0.5)
        else:
            distribution_weight = compute_weight(X_train_val, y_train_val,name_method = namemethod,name_function = namefunction)

        clf = Wsvm(C,distribution_weight)
        clf.fit(X_train_val, y_train_val)
        print(clf.predict([X_test_val[5]]))
        pred2 = clf.predict(X_test_val)    #X_val
        sp,se,gmean = Gmean(y_test_val,pred2) #y_val
        
        metr(X_train_val,y_test_val,pred2,sp,se,gmean)
        #metr_text(f,X_train_val,y_test_val,pred2,sp,se,gmean)
        
        # if ((gmean2 - gmean) <= thamso1) or (gmean2 > thamso2):
        #     f.write("\n_____Gmean_ERROR!!!____\n")
        #     print("\n_____Gmean_ERROR!!!____\n")
        #     return X_train_val, y_train_val
        # else:
        #     gmean = gmean2
        #f.write(f"\n===================================================================================================================\n")
    return X_train_val, y_train_val

In [30]:
def metr(X_train,y_test,test_pred,se,sp,gmean):
    #se,sp,gmean = Gmean(y_test,test_pred)
    print("So luong samples: ",len(X_train))
    print("\n",classification_report(y_test, test_pred))
    print("SP      : ",sp)
    print("SE      : ",se)
    print("Gmean   : ",gmean)
    print("F1 Score: ",f1_score(y_test, test_pred))
    print("Accuracy: ",accuracy_score(y_test,test_pred))
    print("AUC     : ",roc_auc_score(y_test, test_pred))
    print("Ma tran nham lan: \n",confusion_matrix(y_test, test_pred))

In [31]:
def svm_lib(X_train, y_train,X_test):
    svc=SVC(probability=True, kernel='linear')
    model = svc.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return y_pred

In [32]:
C = 100
thamso1 = 1
thamso2 = 1
T = 2
n_neighbors = 5
test_size = [0.2,0.3,0.4]
testsize_val = 0.2
# data = [Co_Author, Abanole, Ecoli, Ecloli1, Ecoli3, Glass1, Glass4, Haberman, Waveform, New_thyroid2, Page_blocks,
#             Pima_Indians_Diabetes, Satimage, Transfusion, Yeast]
data = [Ecoli]
#name_method =["estimated_hyper_lin","actual_hyper_lin","own_class_center_opposite"]
name_method =["own_class_center_divided"]
name_function = ["exp","lin_center_own"]

In [33]:
X_train_val, y_train_val, X_test_val, y_test_val, X_test, y_test = Haberman.load_data(test_size=0.3,testsize_val=0.2)
distribution_weight = compute_weight(X_train_val, y_train_val)
#pred = wsvm(C,X_train_val, y_train_val, X_test, distribution_weight)
#print(pred),
X_train_tl, y_train_tl,_ = data_tomelinks(C,X_train_val,y_train_val,X_test_val, y_test_val, n_neighbors)
for namemethod in name_method:
    for namefunction in name_function:
        print(f"\n\n\tFuzzy W.SVM name_method = '{namemethod}',name_function = '{namefunction}' starting...\n")
        X_train_new, y_train_new = lfb(C,namemethod,namefunction,100, T, X_train_tl,y_train_tl,X_test_val, y_test_val, n_neighbors, thamso1,thamso2) 
        print("DONE!")
        if namefunction == "exp":
            distribution_weight = compute_weight(X_train_new, y_train_new,name_method = namemethod,name_function = namefunction,beta = 0.5)
            print("exp")
        else:
            distribution_weight = compute_weight(X_train_new, y_train_new,name_method = namemethod,name_function = namefunction)
            print("Ko exp")
        test_pred_tl = wsvm(C,X_train_new, y_train_new, X_test, distribution_weight)
        sp,se,gmean = Gmean(y_test, test_pred_tl)
        metr(X_train_new, y_test, test_pred_tl, sp, se, gmean)
        # se,sp,gmean = Gmean(y_test,pred)

        # metr(X_train_val,y_test,pred,se,sp,gmean)
# # ##
# pred2 = svm_lib(X_train_val, y_train_val,X_test)
# se2,sp2,gmean2 = Gmean(y_test,pred2)
# print("SVM LIBRA")
# metr(X_train_val,y_test,pred2,se2,sp2,gmean2)


AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

W mac dinh



	Fuzzy W.SVM name_method = 'own_class_center_divided',name_function = 'exp' starting...

		 Vong Lap thu: T = 1

So luong mau training ban dau: 100


	Fuzzy W.SVM name_method = 'own_class_center_divided',name_function = 'exp' starting...

161
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
BETA:  0.5
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
[1.]
So luong samples:  154

               precision    recall  f1-score   support

        -1.0       0.84      0.97      0.90        32
         1.0       0.83      0.45      0.59        11

    accuracy                           0.84        43
   macro avg       0.84      0.71      0.74        43
weighted avg       0.84      0.84      0.82        43

SP      :  0.96875
SE      :  0.45454545454545453
Gmean   :  0.663581878211656
F1 Score:  0.5882352941176471
Accuracy:  0.8372093023255814
AUC     :  0.7116477272727273
Ma tran nham lan: 
 [[31  1]
 [ 6  5]]
		 Vong Lap thu: T

In [34]:
X_train_val, y_train_val, X_test_val, y_test_val, X_test, y_test = Haberman.load_data(test_size=0.3,testsize_val=0.2)

In [35]:
a = [1.33055036e-01,1.93256405e-01,8.46084996e-02,9.85577407e-02
,7.96895456e-02,1.27073435e-01,9.27549131e-02,1.89629000e-01
,1.08111890e-02,6.47346953e-02,1.28877815e-05,3.97741828e-02
,1.06404151e-01,4.28457738e-02,6.06627497e-02,2.11256646e-01
,5.36435647e-02,6.22377880e-02,1.11848432e-01,3.23473302e-02
,1.28507204e-01,1.45773927e-01,1.66158203e-01,6.13615969e-02
,9.76827180e-02,1.72950670e-01,6.25965561e-02,6.75673283e-02
,1.97751989e-01,1.28877815e-05,8.30779970e-02,4.90421633e-02
,1.45621320e-01,2.13640597e-01,3.75383181e-02,2.79424730e-02
,4.37741111e-02,8.11388228e-02,1.71303364e-01,3.33532377e-02
,7.20919354e-02,6.87193247e-03,6.36242832e-02,1.61821630e-01
,1.28168402e-01,3.53577897e-01,1.60653967e-01,1.53248259e-01
,1.58413159e-01,1.68008902e-01,1.70503123e-01,1.62172887e-01
,1.55441986e-01,1.60363729e-01,1.60133726e-01,1.57614649e-01
,1.61399541e-01,1.62897180e-01,1.57965419e-01,1.67537178e-01
,1.35493029e-01,1.61060379e-01,1.67024782e-01,1.64162494e-01
,1.50100637e-01,1.61895991e-01,1.60787421e-01,1.43440819e-01
,1.66859123e-01,1.61063276e-01,1.65517066e-01,1.53369570e-01
,1.56916030e-01,1.47864485e-01,1.63929393e-01,1.64046322e-01
,1.63657759e-01,1.38879313e-01,1.69651894e-01,1.69742989e-01
,1.60438207e-01,1.56930969e-01,1.61614628e-01,1.55043116e-01
,1.55576971e-01,1.64892436e-01,1.71251448e-01,1.55584613e-01
,1.61779767e-01,1.67121407e-01,1.57254511e-01,1.56697578e-01
,1.62998742e-01,1.51318301e-01,1.55900624e-01,1.59115620e-01
,1.65245780e-01,1.66940509e-01,1.67049737e-01,1.62751039e-01
,1.59291410e-01,1.56485386e-01,1.64255944e-01,1.61063276e-01
,1.47265687e-01,1.58665056e-01,1.56902914e-01,1.61063276e-01
,1.60758193e-01,1.67233789e-01,1.58963596e-01,1.63478432e-01
,1.68559237e-01,1.44938482e-01,1.56822154e-01,1.68624785e-01
,1.63950409e-01,1.57381718e-01,1.59105683e-01,1.60663889e-01
,1.61143961e-01,1.66517834e-01,1.58467006e-01,1.56288332e-01
,1.59925351e-01,1.57697474e-01,1.62950309e-01,1.58187375e-01
,1.68465301e-01,1.60229111e-01,1.59895857e-01,1.71381867e-01
,1.54097851e-01,1.64149711e-01,1.65457750e-01,1.72386233e-01
,1.39139314e-01,1.70503123e-01,1.55311220e-01,3.31842152e-06
,1.61198458e-01,1.52327022e-01,1.62563452e-01,1.61394091e-01
,1.55711533e-01,1.72588445e-01,1.57965419e-01,1.55277397e-01
,1.66940509e-01,1.53571353e-01,1.45246122e-01,1.50451351e-01
,1.50075386e-01,1.58509082e-01,1.62864875e-01,1.57934165e-01
,1.58347444e-01,1.54207189e-01,1.72929353e-01,1.62227270e-01
,1.57781957e-01,1.71017370e-01,1.60154206e-01,1.58619705e-01
,1.57781957e-01,1.71017370e-01,1.60154206e-01,1.58619705e-01,
1.57781957e-01,1.71017370e-01,1.60154206e-01]

In [36]:
cvv = Wsvm(C,distribution_weight)
print(cvv)
cvv.fit(X_train_val, y_train_val)
cvv.predict(X_test)

ValueError: operands could not be broadcast together with shapes (153,) (171,) 